# Preguntas y respuestas 

# ENTREGABLE 4

## Nota! Ejecutar primera celda para disponibilizar datos de los csv que se usan en las posteriores consultas


In [2]:
import pandas as pd
sales_df = pd.read_csv(r'data/sales.csv')
products_df = pd.read_csv(r'data/products.csv')
employees_df = pd.read_csv(r'data/employees.csv')
categories_df=pd.read_csv(r'data/categories.csv')
customers_df = pd.read_csv(r'data/customers.csv')

Crear dataframe

In [ ]:
import utils.pandas_utils as p_utils

sales_data= sales_df.copy()
products_data = products_df.copy()

sales_with_price = sales_data.merge(products_data[['ProductID', 'Price']], on='ProductID', how='left') # agrega el precio del producto
sales_with_price['AggregatedTotalPrice'] = sales_with_price['Quantity'] * sales_with_price['Price'] * (1 - sales_with_price['Discount'])
sales_with_price['isOutlier'] = p_utils.is_outlier(sales_with_price, 'AggregatedTotalPrice',1.5)
sales_with_price['SalesDate'] = pd.to_datetime(sales_with_price['SalesDate'], errors='coerce')
sales_with_price['SalesTime'] = sales_with_price['SalesDate'].dt.hour

sales_with_price['SalesDay'] = sales_with_price['SalesDate'].dt.day_name()
sales_by_day = sales_with_price['SalesDay'].apply(lambda x: 'Weekday' if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] else 'Weekend')
sales_with_price['SalesDay'] = sales_by_day

sales_with_price = sales_with_price.merge(products_df[['ProductID', 'CategoryID', 'IsAllergic']], on='ProductID', how='left')
sales_with_price = sales_with_price.merge(categories_df[['CategoryID', 'CategoryName']], on='CategoryID', how='left')
sales_with_price = sales_with_price.merge(customers_df[['CustomerID', 'CityID']], on='CustomerID', how='left')

sales_dataframe = sales_with_price.drop(columns=['TotalPrice','TransactionNumber'])

sales_dataframe = pd.get_dummies(sales_dataframe, columns=['IsAllergic'], prefix='IsAllergic')
cols = [col for col in sales_dataframe.columns if col.startswith('IsAllergic_')]
sales_dataframe[cols] = sales_dataframe[cols].astype(int)

sales_dataframe['HadDiscount'] = (sales_dataframe['Discount'] > 0).astype(int)

sales_dataframe['onWeekendSale'] = sales_dataframe['SalesDayGroup'].map({
    'Weekday': 0,
    'Weekend': 1
})

sales_dataframe = pd.get_dummies(sales_dataframe, columns=['CategoryName'], prefix='Category')
cols = [col for col in sales_dataframe.columns if col.startswith('Category_')]
sales_dataframe[cols] = sales_dataframe[cols].astype(int)

sales_dataframe.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,SalesDate,Price,AggregatedTotalPrice,isOutlier,SalesTime,SalesDay,CategoryID,CategoryName,CityID,IsAllergic_FALSE,IsAllergic_TRUE,IsAllergic_Unknown,HadDiscount
0,1,6,27039,381,7,0.0,2018-02-05 07:38:25.430,44.2337,309.63590,False,7.0,Weekday,1.0,Confections,54,0,0,1,0
1,2,16,25011,61,7,0.0,2018-02-02 16:03:31.150,62.5460,437.82200,False,16.0,Weekday,8.0,Grain,71,1,0,0,0
2,3,13,94024,23,24,0.0,2018-05-03 19:31:56.880,79.0184,1896.44160,False,19.0,Weekday,11.0,Produce,2,0,1,0,0
3,4,8,73966,176,19,0.2,2018-04-07 14:43:55.420,81.3167,1236.01384,False,14.0,Weekend,6.0,Seafood,45,0,1,0,1
4,5,10,32653,310,9,0.0,2018-02-12 15:37:03.940,79.9780,719.80200,False,15.0,Weekday,9.0,Poultry,82,1,0,0,0


Agrupadito 

In [ ]:
sales_dataframe['SalesDateOnly'] = sales_dataframe["SalesDate"].dt.date

daily_sales = sales_dataframe.groupby("SalesDateOnly")["Aggregated"].sum().sort_index()

# encontrar periodo de 5 días consecutivos con el mayor volumen de ventas totale

# ordenar dataframe por fecha